# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import json
import os
import glob
from pathlib import Path
#from jax import numpy as np
import numpy as np
import numpy
import pickle

from discopy.utils import loads
from pytket.extensions.qiskit import AerBackend
from pytket.extensions.qulacs import QulacsBackend
from pytket.extensions.cirq import CirqStateSampleBackend

backend = QulacsBackend()

from discopy.quantum import Circuit
from discopy.tensor import Tensor

import jax
from jax import jit
from noisyopt import minimizeSPSA

this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

## Read circuit data

We read the circuits from the pickled files.

In [2]:
training_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//training//[0-9]*.p")
#validation_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//validation//[0-9]*.p")
test_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//test//[0-9]*.p")

def read_diagrams(circuit_paths):
    circuits = {}
    for serialized_diagram in circuit_paths:
        base_name = Path(serialized_diagram).stem
        f = open(serialized_diagram, "rb")
        diagram = pickle.load(f)
        circuits[base_name] = diagram
    return circuits


training_circuits = read_diagrams(training_circuits_paths[:2])
#for key in training_circuits:
#    print("training query: ", key)
test_circuits = read_diagrams(test_circuits_paths) #+ [test_circuits_paths[2]] + test_circuits_paths[8:])
#test_circuits = read_diagrams([test_circuits_paths[2]])
#for key in test_circuits:
#    print("test query: ", key)

## Read training and test data

In [3]:
training_data, test_data = None, None
with open(this_folder + "//data//training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(this_folder + "//data//test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
    

def time_to_states(data, circuits):
    labeled_data = {}
    for elem in data:
        if elem["name"] in circuits.keys():
            if elem["time"] < 5000:
                labeled_data[elem["name"]] = [1,0] # corresponds to |0>
            else:
                labeled_data[elem["name"]] = [0,1] # corresponds to |1>
    return labeled_data


training_data_labels = time_to_states(training_data, training_circuits)
test_data_labels = time_to_states(test_data, test_circuits)

#for key in training_data_labels:
#    print("training: ", key)
#for key in test_data_labels:
#    print("test ", key)

## Lambeq optimizer

In [4]:
#all_circuits = list(training_circuits.values()) + list(test_circuits.values())

training_circuits_l = []
test_circuits_l = []
training_data_labels_l = []
test_data_labels_l = []

# Organize circuits and labels in correct order into two lists which will be input for training the model
for key in training_data_labels:
    training_circuits_l.append(training_circuits[key])
    training_data_labels_l.append(training_data_labels[key])

for key in test_data_labels:
    test_circuits_l.append(test_circuits[key])
    test_data_labels_l.append(test_data_labels[key])
    
all_circuits = training_circuits_l + test_circuits_l

train_syms = set([sym for circuit in training_circuits.values() for sym in circuit.free_symbols])
test_syms = set([sym for circuit in test_circuits.values() for sym in circuit.free_symbols])

print("Test circuits need to share training circuits' parameters. The parameters that are not covered: ", test_syms.difference(train_syms))

print("Total number of circuits: ", len(all_circuits))
print("Total number of variables: ", len(train_syms))

Test circuits need to share training circuits' parameters. The parameters that are not covered:  {'bottom 10 rank'__n.l_0, link__n.l_1, cast_info__n.l_2, 'Sweden'__n.l_0, '%(presents)%'__n.l_1, <_n.l@n.l_n.l_0, aka_title__n.l_0, aka_title__n.l_1, production_year__n.l_1, movie_info__n.l_1, kind__n.l_1, '[us]'__n.l_1, info_type_id__n.l_2, linked_movie_id__n.l_1, FROM_n@n.l@n.l_n_0, NULL__n.l_1, 'USA'__n.l_1, info__n.l_0, 'German'__n.l_2, 'B%'__n.l_0, NOT_n.l_n.l_0, aka_name__n.l_2, 'features'__n.l_2, 'top 250 rank'__n.l_0, keyword__n.l_1, 'movie'__n.l_0, complete_cast__n.l_1, link__n.l_2, kind_id__n.l_2, link_type_id__n.l_0, person_info__n.l_2, set_n.l@n.l@n.l_n.l_0, role_id__n.l_2, 2011__n.l_1, kind_type__n.l_1, info_type_id__n.l_0, role_id__n.l_0, 'production companies'__n.l_2, '%(as Metro-Goldwyn-Mayer Pictures)%'__n.l_1, char_name__n.l_0, person_role_id__n.l_0, '%sequel%'__n.l_2, NOT_n.l_n.l_2, 'Queen'__n.l_2, '(voice: English version)'__n.l_0, 'top 250 rank'__n.l_2, keyword_id__n.l_

In [5]:
training_data_labels_l = np.array(training_data_labels_l)
test_data_labels_l = np.array(test_data_labels_l)

## Model

In [6]:
from sympy import default_sort_key

parameters = sorted(
    train_syms,
    key=default_sort_key)

len(parameters)

41

In [7]:
def make_pred_fn(circuits):
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

train_pred_fn = make_pred_fn(training_circuits_l)
#dev_pred_fn = jit(make_pred_fn(dev_circuits))
test_pred_fn = make_pred_fn(test_circuits_l)

## Loss function and evaluation

In [8]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)
        #print(labels)
        #print(predictions)

        cost = -np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        acc = np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(acc)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Trainer

In [9]:
EPOCHS = 1000
SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

import numpy

rng = numpy.random.default_rng(SEED)
init_params_spsa = np.array(rng.random(len(parameters)))
numpy.random.seed(SEED)

#print(jax.make_jaxpr(train_pred_fn)(init_params_spsa))

train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
#dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, dev_labels)

def callback_fn(xk):
    #cost_val = train_cost_fn(xk, costs = cost_store_spsa, accuracies = acc_store_spsa)
    cost_val = train_costs[-1]
    acc_val = train_accs[-1]
    iters = len(train_accs)/2
    print(
            #f"Params = {xk}, "
            f"Iteration = {iters}, "
            f"Cost = {cost_val}",
            f"Accuracy on training data = {acc_val}"
        )

In [10]:
result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a=1, c=0.12, niter=EPOCHS, callback=callback_fn)

AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
import matplotlib.pyplot as plt

fig, ((ax_tl, ax_tr), (ax_bl, ax_br)) = plt.subplots(2, 2, sharex=True, sharey='row', figsize=(10, 6))
ax_tl.set_title('Training set')
ax_tr.set_title('Development set')
ax_bl.set_xlabel('Iterations')
ax_br.set_xlabel('Iterations')
ax_bl.set_ylabel('Accuracy')
ax_tl.set_ylabel('Loss')

colours = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
ax_tl.plot(train_costs[1::2], color=next(colours))  # training evaluates twice per iteration
ax_bl.plot(train_accs[1::2], color=next(colours))   # so take every other entry
#ax_tr.plot(dev_costs, color=next(colours))
#ax_br.plot(dev_accs, color=next(colours))

# print test accuracy
test_cost_fn, _, test_accs = make_cost_fn(test_pred_fn, test_data_labels_l)
test_cost_fn(result.x)
print('Test accuracy:', test_accs[0])